In [2]:
import cudf
import cuspatial
import geopandas
import cupy as cp

In [5]:
zones = .read_parquet('rapids-carto-talk/modules/datasets/data/polys.arrow'))
taxi2015 = cudf.read_csv('taxi2015.csv')

ArrowInvalid: Error creating dataset. Could not read schema from '/home/tcomer/mnt/NVIDIA/rapids-docker/cuspatial/notebooks/thomcom_notebooks/GTC 2023/rapids-carto-talk/modules/datasets/data/polys.arrow': Could not open Parquet input source '/home/tcomer/mnt/NVIDIA/rapids-docker/cuspatial/notebooks/thomcom_notebooks/GTC 2023/rapids-carto-talk/modules/datasets/data/polys.arrow': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.. Is this a 'parquet' file?

In [5]:
def make_list_series_from_lon_lat(lon, lat):
    from cudf.core.column import build_list_column, arange
    
    assert len(lon) == len(lat)
    xy = cudf.Series(cp.zeros(len(lon) * 2))
    xy[::2] = lon
    xy[1::2] = lat

    elements = cudf.Series(xy)._column
    indices = arange(0, len(elements) + 1, 2, dtype="int32")
    size = len(lon)
    list_column = build_list_column(
        indices=indices,
        elements=elements,
        size=size
    )
    return cudf.Series(list_column)
    
def make_geoseries_from_list_series(series):    
    from cudf.core.column import build_list_column
    from cuspatial.core._column.geometa import Feature_Enum, GeoMeta
    from cuspatial.core._column.geocolumn import GeoColumn
    geo_col = GeoColumn(
        (
            series,
            cudf.Series(),
            cudf.Series(),
            cudf.Series()
        ),
        GeoMeta({
            "input_types": cp.full(
                len(series),
                Feature_Enum.POINT.value,
            ),
            "union_offsets": cp.arange(len(series))
        })
    )
    return cuspatial.GeoSeries(geo_col)

In [6]:
pickups_cudf = make_list_series_from_lon_lat(
    taxi2015['pickup_longitude'],
    taxi2015['pickup_latitude']
)
pickups = make_geoseries_from_list_series(pickups_cudf)
dropoffs_cudf = make_list_series_from_lon_lat(
    taxi2015['dropoff_longitude'],
    taxi2015['dropoff_latitude']
)
dropoffs = make_geoseries_from_list_series(dropoffs_cudf)
pickups = cuspatial.GeoSeries(cuspatial.core._column.geocolumn.GeoColumn._from_points_xy(taxi2015['pickup_longitude']._column))

In [71]:
%timeit zones["geometry"].iloc[0:1].contains_properly(dropoffs)

498 ms ± 14.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
host_pickups = pickups.to_geopandas()
host_dropoffs = dropoffs.to_geopandas()

In [87]:
host_polys = zones["geometry"].to_geopandas()

In [89]:
host_polys

0      POLYGON ((933100.918 192536.086, 933091.011 19...
1      MULTIPOLYGON (((1033269.244 172126.008, 103343...
2      POLYGON ((1026308.770 256767.698, 1026495.593 ...
3      POLYGON ((992073.467 203714.076, 992068.667 20...
4      POLYGON ((935843.310 144283.336, 936046.565 14...
                             ...                        
258    POLYGON ((1025414.782 270986.139, 1025138.624 ...
259    POLYGON ((1011466.966 216463.005, 1011545.889 ...
260    POLYGON ((980555.204 196138.486, 980570.792 19...
261    MULTIPOLYGON (((999804.795 224498.527, 999824....
262    POLYGON ((997493.323 220912.386, 997355.264 22...
Name: geometry, Length: 263, dtype: geometry

In [78]:
gpu_pip = zones["geometry"].iloc[0:1].contains_properly(dropoffs)
host_pip = zones["geometry"].to_geopandas().iloc[0:1].contains(host_dropoffs)

/home/tcomer/mnt/NVIDIA/rapids-docker/compose/etc/conda/cuda_11.6/envs/notebooks/lib/python3.8/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


In [84]:
(gpu_pip.to_pandas() == host_pip).all()

True